## Imports

In [1]:
import copy
import json
import math
import pprint
import random
import scipy
import sklearn.model_selection
import statistics

## Data loading and processing

In [68]:
ranked_data_path = f"bigresponse_20241102.json"
ranked_data_file = open(ranked_data_path,encoding="utf-8")
ranked_data = json.load(ranked_data_file)

In [69]:
players = ranked_data["players"]
maps = ranked_data["maps"]
scores = ranked_data["scores"]
filtered_scores = [score for score in scores if score["accuracy"] < 100]

In [70]:
players_by_id = {player["id"] : player for player in players}
maps_by_id = {bmap["id"] : bmap for bmap in maps}

In [71]:
scores_by_player_id = {}
scores_by_map_id = {}

def add_score(score):
    player_id = score["playerId"]
    map_id = score["leaderboardId"]
    
    player_map = scores_by_player_id.get(player_id,{})
    player_map[map_id] = score
    
    scores_by_player_id[player_id] = player_map
    
    map_map = scores_by_map_id.get(map_id,{})
    map_map[player_id] = score
    
    scores_by_map_id[map_id] = map_map
    
#for score in scores:
for score in filtered_scores:
    add_score(score)

In [72]:
modifiers = {"SF":1.2,
             "GN":1,
             "SA":1.05,
             "PM":1,
             "IF":1,
             "NO":0.5,
             "BE":1,
             "SS":0.75,
             "FS":1.05,
             "NB":0.5,
             "SC":1,
             "OD":1,
             "DA":1,
             "CS":1,
             "NA":0.5,
             "OP":0.5}
"""
"SF" - Super Fast,
"GN" - Ghost Notes,
"SA" - Strict Angles,
"PM" - Pro Mode,
"IF" - 1 life,
"NO" - No obstacles (no walls),
"BE" - Battery Energy,
"SS" - Slow song,
"FS" - Fast song,
"NB" - No bombs,
"SC" - Small Notes,
"OD" - Old Dots,
"DA" - Disappearing Arrows,
"CS" - ???,
"NA" - No Arrows -50%,
"OP" - Out of platform -50% 
"""

def modified_score(score,score_modifiers):
    final_score = score
    for modifier,multiplier in modifiers.items():
        if modifier in score_modifiers:
            if multiplier > 1:
                final_score = 1 - (1 - final_score)*(2 - multiplier)
            else:
                final_score = final_score * multiplier
            
    return final_score

## Mathematical functions for transforming and combining numerical values

In [7]:
"""
This is a mapping on the probability space rather than the population space.
That is, from (0,1) to (0,1). A probability distribution on the (0,1) interval.
We can do this with the Beta distribution, though other distributions might work.

I approximate a value for alpha and beta using this: https://homepage.divms.uiowa.edu/~mbognar/applets/beta.html
Trying to get the median around 0.9 and the probability close to 0 for 0.5, but a little bit higher for 0.65ish.
Beat Saber scores are typically at least 65%, on median probably around 0.9. They also must go down as the value
approaches 1, so beta must be greater than 1.

alpha = 10 and beta = 1.25 seemed to work quite well.
"""
beta_alpha = 10
beta_beta = 1.25

def beta_value(perc_value, beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.cdf(perc_value, beta_alpha, beta_beta)

def perc_beta_value(beta_value,beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.ppf(beta_value, beta_alpha, beta_beta)

In [8]:
"""
There are some issues with this, like that a 100% score could lower a player's skill level
if it is on a very easy map. But this is very unlikely with the numbers, and 100% scores
are exceedingly rare anyway. There are also ways to deal with that later after estimating
difficulties.
"""

# truncexp_max = 25
truncexp_max = 100
# truncexp_base_mean = 7
truncexp_base_mean = 10

def truncexp_value(perc_value,base_mean=truncexp_base_mean,maxx=truncexp_max,beta_alpha=beta_alpha,beta_beta=beta_beta):
    return scipy.stats.truncexpon.ppf(beta_value(perc_value,beta_alpha=beta_alpha,beta_beta=beta_beta),b=maxx,scale=base_mean)

def perc_truncexp_value(truncexp_value,base_mean=truncexp_base_mean,maxx=truncexp_max,beta_alpha=beta_alpha,beta_beta=beta_beta):
    return perc_beta_value(scipy.stats.truncexpon.cdf(truncexp_value,b=maxx,scale=base_mean),beta_alpha=beta_alpha,beta_beta=beta_beta)

In [9]:
"""
We normalize around a value of 7.
This means that a X star player on a 7 accability map achieves an X star score
and a 7 star player on an X accability map achieves an X star score.
"""

#linear_mean = 7
linear_mean = 10

def score_linear(skill,accability,linear_mean = linear_mean):
    return skill * accability / linear_mean

def accability_linear(skill,score,linear_mean = linear_mean):
    return score / skill * linear_mean

def skill_linear(accability,score,linear_mean = linear_mean):
    return score / accability * linear_mean

In [10]:
def abs_prop_error(value,evalue):        
    return abs((value-evalue)/evalue)

def abs_error(value,evalue):
    return abs(value-evalue)

In [11]:
"""
Let's start with something simple: Just use the median instead of the average.
"""

def aggregation_median(scores,default_value=1):
    if scores == []:
        return default_value
    else:
        return statistics.median(scores)
    
def aggregation_median_f(default_value):
    def f(scores):
        return aggregation_median(scores,default_value)
    
    return f

In [12]:
"""
Do the average of the top percentage of scores.
"""
def aggregation_topscores(scores,perc=0.25,default_value=1):
    scores.sort()
    l = len(scores)
    n = math.floor(l*perc)
    
    if n == 0:
        return default_value
    else:
        return statistics.mean(scores[l-n:])
    
def aggregation_topscores_f(perc,default_value):
    def f(scores):
        return aggregation_topscores(scores,perc,default_value)
    
    return f

In [13]:
"""
Do the average of the bottom percentage of scores.
"""
def aggregation_bottomscores(scores,perc=0.25,default_value=1):
    scores.sort()
    l = len(scores)
    n = math.floor(l*perc)
    
    if n == 0:
        return default_value
    else:
        return statistics.mean(scores[0:n])
    
def aggregation_bottomscores_f(perc,default_value):
    def f(scores):
        return aggregation_bottomscores(scores,perc,default_value)
    
    return f

## Main algorithm

In [16]:
class BiPartiteStabilizer:
    """No description yet"""
    
    def __init__(self,
                 anodes_ratings,bnodes_ratings,
                 wdata,afun,bfun,wfun,
                 aggregation_fun=aggregation_median,
                 default_rating=1,
                 max_iter=50,
                 error_fun=abs_error,
                 error_aggregation_fun=aggregation_bottomscores,
                 finish_early=True,
                 error_change_prop=0.001):
        """
        anodes_ratings and bnodes_ratings must be dictionaries with the node identifiers as keys
        and initial ratings as values.
        
        wdata must be a doubly indexed dictionary with anode identifiers and bnode identifiers as respective indexes,
        respectively, and weights as values.
        
        afun and bfun must be functions taking a value of the other node type as first argument
        and a weight as the second argument, that returns the corresponding value for the node.
        
        Similarly, wfun must be a function that takes the value of an anode and bnode and returns the
        correct weight.
        
        These functions must be such that:
        - wfun(afun(b,w),b) = w
        - wfun(a,bfun(a,w)) = w
        - afun(bfun(a,w),w) = a
        - afun(b,wfun(a,b)) = a
        - bfun(afun(b,w),w) = b
        - bfun(a,wfun(a,b)) = b
        
        error_fun must take two parameters (actual value, expected value) and return a number indicating the error for that
        particular edge
        """
        self.anodes_ratings = anodes_ratings
        self.bnodes_ratings = bnodes_ratings
        
        self.adata = wdata
        self.bdata = self.process_bdata(wdata)
        
        self.afun = afun
        self.bfun = bfun
        self.wfun = wfun
        
        self.aggregation_fun = aggregation_fun
        
        self.error_fun = error_fun
        self.error_aggregation_fun = error_aggregation_fun
        
        self.default_rating = default_rating
        
        self.average_error = math.inf
        
        self.iter = 0
        self.max_iter = max_iter
        
        # Finish early when error increases
        self.finish_early = finish_early
        self.last_average_error = math.inf
        self.last_anodes_ratings = anodes_ratings
        self.last_bnodes_ratings = bnodes_ratings       
    
        self.error_change_prop = error_change_prop
    
    def process_bdata(self, wdata):
        bdata = {}
        
        for anode_id, anode_data in wdata.items():
            for bnode_id, w in anode_data.items():
                bnode_data = bdata.get(bnode_id,{})
                bnode_data[anode_id] = w
                bdata[bnode_id] = bnode_data
        
        return bdata       
        
    def acycle(self):
        for anode_id in self.anodes_ratings:
            if anode_id in self.adata:
                self.anode_process(anode_id)           
            
    def anode_process(self,anode_id):
        anode_data = self.adata.get(anode_id,False)
                
        calc_values = []
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            # asum += clamp(self.afun(bnode_value,w))
            # asum += self.afun(bnode_value,w)
            calc_values.append(self.afun(bnode_value,w))
        
        avg = self.aggregation_fun(calc_values)
        
        # self.anodes_ratings[anode_id] = clamp(avg)
        self.anodes_ratings[anode_id] = avg
    
    def bcycle(self):
        for bnode_id in self.bnodes_ratings:
            if bnode_id in self.bdata:
                self.bnode_process(bnode_id)                
    
    def bnode_process(self,bnode_id):
        bnode_data = self.bdata[bnode_id]
        
        calc_values = []
        for anode_id, w in bnode_data.items():
            anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
            # bsum += clamp(self.bfun(anode_value,w))
            calc_values.append(self.bfun(anode_value,w))
        
        avg = self.aggregation_fun(calc_values)
        
        # self.bnodes_ratings[bnode_id] = clamp(avg)
        self.bnodes_ratings[bnode_id] = avg
    
    def save_last(self):
        self.last_anodes_ratings = self.anodes_ratings
        self.last_bnodes_ratings = self.bnodes_ratings
        self.last_average_error = self.average_error
        
    def restore_last(self):
        self.anodes_ratings = self.last_anodes_ratings
        self.bnodes_ratings = self.last_bnodes_ratings
        self.average_error = self.last_average_error
    
    def iterate(self):
        self.save_last()
        
        self.bcycle()
        self.acycle()
        
        self.average_error = self.measure_error(self.adata)
        
        self.iter += 1        
        print(f"Iteration {self.iter}, Average error: {self.average_error}")         
        
    def run(self):
        while (self.iter <= self.max_iter):            
            self.iterate()
            
            # Finish early with previous result if error went up
            if self.finish_early and (self.average_error > self.last_average_error):
                self.restore_last()
                print(f"Finishing early due to increased average error. Restoring previous values.")
                break
                
            # Finish if the error did not change by more than the error_change_prop
            if self.last_average_error != math.inf:
                abs_error_change = abs(self.average_error - self.last_average_error)
                if (abs_error_change / self.last_average_error) < self.error_change_prop:
                    print(f"Finishing due to change in average error less than {self.error_change_prop} (proportional)")
                    break
                
        return (self.anodes_ratings,self.bnodes_ratings)
    
    def measure_error(self,wdata_test):
        calc_values = []
        for anode_id in wdata_test:
            anode_data = wdata_test[anode_id]
            anode_value = self.anodes_ratings.get(anode_id,self.default_rating)

            for bnode_id, w in anode_data.items():
                bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
                calculated_w = self.wfun(anode_value,bnode_value)
                error = self.error_fun(calculated_w,w)
                calc_values.append(error)
            
        average_error = self.error_aggregation_fun(calc_values)
        
        return average_error      
     

## Prepare scores in the right format

In [73]:
scores_doubly_indexed = {map_id : {player_id : truncexp_value(modified_score(scores_by_map_id[map_id][player_id]["accuracy"],scores_by_map_id[map_id][player_id]["modifiers"])) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}

In [15]:
test_set_size = 0.2

scores_doubly_indexed_training_items, scores_doubly_indexed_test_items = sklearn.model_selection.train_test_split(list(scores_doubly_indexed.items()),test_size=test_set_size)
scores_doubly_indexed_training = dict(scores_doubly_indexed_training_items)
scores_doubly_indexed_test = dict(scores_doubly_indexed_test_items)

## Run the algorithm

In [74]:
#default_rating = 7
default_rating = 10

map_ratings = {map_id : default_rating for map_id in scores_by_map_id}
player_ratings = {player_id: default_rating for player_id in scores_by_player_id}

# aggregation_fun = aggregation_median_f(default_rating)
aggregation_topscores_p = 0.9
error_aggregation_bottomscores_p = 0.5
aggregation_fun = aggregation_topscores_f(aggregation_topscores_p,default_rating)
error_aggregation_fun=aggregation_bottomscores_f(error_aggregation_bottomscores_p,0)

bps = BiPartiteStabilizer(map_ratings,player_ratings,scores_doubly_indexed,
#bps = BiPartiteStabilizer(map_ratings,player_ratings,scores_doubly_indexed_training,
                          accability_linear,skill_linear,score_linear,
                          aggregation_fun=aggregation_fun,
                          default_rating = default_rating,
                          error_aggregation_fun=error_aggregation_fun,
                          max_iter=50,
                          finish_early=True,
                          error_change_prop=0.001)

#bps.iterate()
#(maps_1,players_1) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))
#maps_1_errors = {key:bps.anode_error(key) for key in maps_1}
#bps.iterate()
#(maps_2,players_2) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))

(map_ratings,player_ratings) = bps.run()

Iteration 1, Average error: 0.7786625533143048
Iteration 2, Average error: 0.695926330485894
Iteration 3, Average error: 0.675855187439784
Iteration 4, Average error: 0.6690494865550328
Iteration 5, Average error: 0.666358681529282
Iteration 6, Average error: 0.6651149471693183
Iteration 7, Average error: 0.6645072741546199
Finishing due to change in average error less than 0.001 (proportional)


## Basic measurement of error

In [19]:
print(bps.measure_error(scores_doubly_indexed_test))

2.0931979824250972


## Outputting learned ratings

In [93]:
with open('map_ratings_20241104_newscores.txt', 'w') as f:
    pprint.pprint(map_ratings, width=1, stream=f)
    
with open('player_ratings_20241104_newscores.txt', 'w') as f:
    pprint.pprint(player_ratings, width=1, stream=f)

## Hyper-parameter search

In [37]:
# Reset the hyper-parameter search result list
# Problem variations. Each of these will have a best result. But they need compared manually.
problem_grid = {
    "error_aggregation_bottomscores_p": [0.25,0.5,0.75,0.9]
}

hs_results = []
for hs_error_aggregation_bottomscores_p in problem_grid["error_aggregation_bottomscores_p"]:
    hs_results.append({
        "problem_params":{"error_aggregation_bottomscores_p":hs_error_aggregation_bottomscores_p},
        "runs":[]
    })

In [38]:
# Fixed parameters
default_rating = 10
truncexp_base_mean = default_rating
linear_mean = default_rating
test_set_size = 0.2
crossvalidation_splits = 5

# Hyperparameter grid. Some may only have one value but in principle it makes sense to explore more.
hyperparam_grid = {
    "beta_alpha": [10],
    "beta_beta": [1.25],
    "truncexp_max": [100],
    "aggregation_topscores_p": [0.25,0.5,0.75,0.9],    
    "max_iter":[50],
    "finish_early": [False],
    "error_change_prop": [0.005,0.001,0.0002]    
}

def hyperparam_grid_size(hyperparam_grid):
    result = 1
    for key,values in hyperparam_grid.items():
        result = result*len(values)
        
    return result

print(hyperparam_grid_size(hyperparam_grid))

# This hyper-parameter search is performed more or less manually, for two reasons:
# - It is considerably easier to program and make sure it works.
# - It allows us to optimize some of the operations to not have to re-do them when the parameters that affect it
#  have not changed (e.g. truncexp_max affecting data pre-processing)

for hs_error_aggregation_bottomscores_p in problem_grid["error_aggregation_bottomscores_p"]:
    print(f"error_aggregation_bottomscores_p = {hs_error_aggregation_bottomscores_p}")
    
    for hs_beta_alpha in hyperparam_grid["beta_alpha"]:
        print(f"beta-alpha = {hs_beta_alpha}")
        for hs_beta_beta in hyperparam_grid["beta_beta"]:
            print(f"beta-beta = {hs_beta_beta}")
            hs_beta_value = lambda perc_value : beta_value(perc_value,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)
            hs_perc_beta_value = lambda beta_value : perc_beta_value(beta_value,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)

            for hs_truncexp_max in hyperparam_grid["truncexp_max"]:
                print(f"truncexp_max = {hs_truncexp_max}")
                hs_truncexp_value = lambda perc_value : truncexp_value(perc_value,base_mean=truncexp_base_mean,maxx=hs_truncexp_max,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)
                hs_perc_truncexp_value = lambda truncexp_value : perc_truncexp_value(truncexp_value,base_mean=truncexp_base_mean,maxx=hs_truncexp_max,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)

                hs_scores_doubly_indexed = {map_id : {player_id : hs_truncexp_value(modified_score(scores_by_map_id[map_id][player_id]["accuracy"],scores_by_map_id[map_id][player_id]["modifiers"])) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}

                hs_score_linear = lambda skill, accability : score_linear(skill,accability,linear_mean = linear_mean)
                hs_accability_linear = lambda skill, score : accability_linear(skill,score,linear_mean = linear_mean)
                hs_skill_linear = lambda accability, score : skill_linear(accability,score,linear_mean = linear_mean)
               
                for hs_aggregation_topscores_p in hyperparam_grid["aggregation_topscores_p"]:
                    print(f"aggregation_topscores_p = {hs_aggregation_topscores_p}")
                    for hs_max_iter in hyperparam_grid["max_iter"]:
                        print(f"max_iter = {hs_max_iter}")
                        for hs_finish_early in hyperparam_grid["finish_early"]:
                            print(f"finish_early = {hs_finish_early}")
                            for hs_error_change_prop in hyperparam_grid["error_change_prop"]:
                                print(f"error_change_prop = {hs_error_change_prop}")
                                
                                hs_crossvalidation_train_error_mean = 0
                                hs_crossvalidation_test_error_mean = 0                                
                                for hs_crossvalidation_i in range(crossvalidation_splits):
                                    print(f"Crossvalidation run {hs_crossvalidation_i}")                                          

                                    hs_scores_doubly_indexed_training_items, hs_scores_doubly_indexed_test_items = sklearn.model_selection.train_test_split(list(hs_scores_doubly_indexed.items()),test_size=test_set_size)
                                    hs_scores_doubly_indexed_training = dict(hs_scores_doubly_indexed_training_items)
                                    hs_scores_doubly_indexed_test = dict(hs_scores_doubly_indexed_test_items)

                                    hs_map_ratings = {map_id : default_rating for map_id in scores_by_map_id}
                                    hs_player_ratings = {player_id: default_rating for player_id in scores_by_player_id}

                                    hs_aggregation_fun = aggregation_topscores_f(hs_aggregation_topscores_p,default_rating)
                                    hs_error_aggregation_fun = aggregation_bottomscores_f(hs_error_aggregation_bottomscores_p,0)

                                    hs_bps = BiPartiteStabilizer(hs_map_ratings,hs_player_ratings,hs_scores_doubly_indexed_training,
                                                              hs_accability_linear,hs_skill_linear,hs_score_linear,
                                                              aggregation_fun=hs_aggregation_fun,
                                                              default_rating = default_rating,
                                                              error_aggregation_fun=hs_error_aggregation_fun,
                                                              max_iter=hs_max_iter,
                                                              finish_early=hs_finish_early,
                                                              error_change_prop=hs_error_change_prop)

                                    (hs_map_ratings,hs_player_ratings) = hs_bps.run()
                                    
                                    hs_error_train = hs_bps.average_error
                                    hs_error_test = hs_bps.measure_error(hs_scores_doubly_indexed_test)
                                    
                                    hs_crossvalidation_train_error_mean += hs_error_train/crossvalidation_splits
                                    hs_crossvalidation_test_error_mean += hs_error_test/crossvalidation_splits
                                    
                                hs_run = {
                                    "hyperparams":{
                                        "default_rating":default_rating,
                                        "truncexp_base_mean":truncexp_base_mean,
                                        "linear_mean":linear_mean,
                                        "test_set_size":test_set_size,
                                        "crosssvalidation_splits":crossvalidation_splits,
                                        "beta_alpha":hs_beta_alpha,
                                        "beta_beta":hs_beta_beta,
                                        "truncexp_max":hs_truncexp_max,
                                        "aggregation_topscores_p":hs_aggregation_topscores_p,
                                        "max_iter":hs_max_iter,
                                        "finish_early":hs_finish_early,
                                        "error_change_prop":hs_error_change_prop
                                    },
                                    "results":{
                                        "train_error":hs_crossvalidation_train_error_mean,
                                        "test_error":hs_crossvalidation_test_error_mean
                                    }
                                }
                                
                                pprint.pprint(hs_run,width=1)
                                
                                hs_runs_found = False
                                for hs_problem in hs_results:
                                    if hs_problem["problem_params"] == {"error_aggregation_bottomscores_p":hs_error_aggregation_bottomscores_p}:
                                        hs_problem["runs"].append(hs_run)
                                        hs_runs_found = True
                                        break

                                if not hs_runs_found:
                                    raise                                

12
error_aggregation_bottomscores_p = 0.25
beta-alpha = 10
beta-beta = 1.25
truncexp_max = 100
aggregation_topscores_p = 0.25
max_iter = 50
finish_early = False
error_change_prop = 0.005
Crossvalidation run 0
Iteration 1, Average error: 0.7657156911584215
Iteration 2, Average error: 0.6267612435271095
Iteration 3, Average error: 0.5939258825115318
Iteration 4, Average error: 0.5839461498064301
Iteration 5, Average error: 0.5801330036859385
Iteration 6, Average error: 0.5785293494199618
Finishing due to change in average error less than 0.005 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 0.7612358687581248
Iteration 2, Average error: 0.6257566364579079
Iteration 3, Average error: 0.593092901285864
Iteration 4, Average error: 0.5830504197050812
Iteration 5, Average error: 0.579286435397081
Iteration 6, Average error: 0.5777475544331607
Finishing due to change in average error less than 0.005 (proportional)
Crossvalidation run 2
Iteration 1, Average error: 0.76810204072

Iteration 10, Average error: 0.5789022002109042
Finishing due to change in average error less than 0.0002 (proportional)
{'hyperparams': {'aggregation_topscores_p': 0.25,
                 'beta_alpha': 10,
                 'beta_beta': 1.25,
                 'crosssvalidation_splits': 5,
                 'default_rating': 10,
                 'error_change_prop': 0.0002,
                 'finish_early': False,
                 'linear_mean': 10,
                 'max_iter': 50,
                 'test_set_size': 0.2,
                 'truncexp_base_mean': 10,
                 'truncexp_max': 100},
 'results': {'test_error': 0.9502865235487715,
             'train_error': 0.5791491514283763}}
aggregation_topscores_p = 0.5
max_iter = 50
finish_early = False
error_change_prop = 0.005
Crossvalidation run 0
Iteration 1, Average error: 0.4700593721266631
Iteration 2, Average error: 0.4028669442340518
Iteration 3, Average error: 0.38659247831449284
Iteration 4, Average error: 0.380319413171231

Iteration 7, Average error: 0.3760057012644016
Iteration 8, Average error: 0.3755863120163851
Iteration 9, Average error: 0.37533396363162297
Iteration 10, Average error: 0.3751696280783803
Iteration 11, Average error: 0.3750671930827576
Iteration 12, Average error: 0.3750044875108904
Finishing due to change in average error less than 0.0002 (proportional)
Crossvalidation run 3
Iteration 1, Average error: 0.4743155275511604
Iteration 2, Average error: 0.40392763679200516
Iteration 3, Average error: 0.38791968075905386
Iteration 4, Average error: 0.38179111099114066
Iteration 5, Average error: 0.3789089746121039
Iteration 6, Average error: 0.3774529925664273
Iteration 7, Average error: 0.3766802595766122
Iteration 8, Average error: 0.37617983918359005
Iteration 9, Average error: 0.3758712935898376
Iteration 10, Average error: 0.37568063523472706
Iteration 11, Average error: 0.37555640995505346
Iteration 12, Average error: 0.37547506382114226
Iteration 13, Average error: 0.37542075094509

Iteration 1, Average error: 0.3831193134026682
Iteration 2, Average error: 0.33733827275302447
Iteration 3, Average error: 0.3258625912742028
Iteration 4, Average error: 0.3210737592394731
Iteration 5, Average error: 0.3186797952567014
Iteration 6, Average error: 0.31735515544681264
Iteration 7, Average error: 0.31661571428269564
Iteration 8, Average error: 0.31619268696325054
Iteration 9, Average error: 0.3159573978760277
Iteration 10, Average error: 0.31582369301327406
Iteration 11, Average error: 0.31574459754699863
Iteration 12, Average error: 0.3156966531452532
Finishing due to change in average error less than 0.0002 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 0.38272216687482613
Iteration 2, Average error: 0.33755282537704934
Iteration 3, Average error: 0.3266069948711413
Iteration 4, Average error: 0.3217752523442171
Iteration 5, Average error: 0.3193209001155439
Iteration 6, Average error: 0.3179672560471064
Iteration 7, Average error: 0.31721555611651525


Iteration 7, Average error: 0.30012057513902735
Iteration 8, Average error: 0.2999608584110681
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 4
Iteration 1, Average error: 0.3611852010908505
Iteration 2, Average error: 0.3214624488677925
Iteration 3, Average error: 0.31039520461829645
Iteration 4, Average error: 0.3058295804080622
Iteration 5, Average error: 0.30368796452526753
Iteration 6, Average error: 0.3025775494433666
Iteration 7, Average error: 0.3021128172133155
Iteration 8, Average error: 0.3019829647262212
Finishing due to change in average error less than 0.001 (proportional)
{'hyperparams': {'aggregation_topscores_p': 0.9,
                 'beta_alpha': 10,
                 'beta_beta': 1.25,
                 'crosssvalidation_splits': 5,
                 'default_rating': 10,
                 'error_change_prop': 0.001,
                 'finish_early': False,
                 'linear_mean': 10,
                 'max_iter': 50,
 

...

Iteration 1, Average error: 1.2760647585866598
Iteration 2, Average error: 1.138339019086504
Iteration 3, Average error: 1.1086371521880323
Iteration 4, Average error: 1.10008334614265
Iteration 5, Average error: 1.0973159844505624
Iteration 6, Average error: 1.0963885660082713
Iteration 7, Average error: 1.0960808644882447
Iteration 8, Average error: 1.0960016675872457
Finishing due to change in average error less than 0.0002 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 1.2763091309932786
Iteration 2, Average error: 1.1404055248219267
Iteration 3, Average error: 1.1118864941745563
Iteration 4, Average error: 1.1039104381203313
Iteration 5, Average error: 1.1013906123333577
Iteration 6, Average error: 1.1005662971588517
Iteration 7, Average error: 1.1003190368159455
Iteration 8, Average error: 1.1002732745415926
Finishing due to change in average error less than 0.0002 (proportional)
Crossvalidation run 2
Iteration 1, Average error: 1.2810503477884585
Iteration 2, A

Iteration 12, Average error: 2.4994235451702336
Iteration 13, Average error: 2.5006292006889543
Iteration 14, Average error: 2.501581818529261
Iteration 15, Average error: 2.5023410804804147
Iteration 16, Average error: 2.502953402998817
Iteration 17, Average error: 2.5034547140939214
Iteration 18, Average error: 2.503872875278558
Finishing due to change in average error less than 0.0002 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 3.1784389051779645
Iteration 2, Average error: 2.642109275861941
Iteration 3, Average error: 2.529058129675436
Iteration 4, Average error: 2.5000498200840844
Iteration 5, Average error: 2.4942913451768614
Iteration 6, Average error: 2.4955232808198855
Iteration 7, Average error: 2.498706108741728
Iteration 8, Average error: 2.5020952221200274
Iteration 9, Average error: 2.5051334838215387
Iteration 10, Average error: 2.5076760778458485
Iteration 11, Average error: 2.509737649498626
Iteration 12, Average error: 2.5113889300946037
Iteration

Iteration 2, Average error: 1.8946026954145767
Iteration 3, Average error: 1.8408523873886555
Iteration 4, Average error: 1.8326525765412738
Iteration 5, Average error: 1.834607160747879
Iteration 6, Average error: 1.8384286848046927
Iteration 7, Average error: 1.841960008446018
Iteration 8, Average error: 1.8447591606928218
Iteration 9, Average error: 1.8468371995890505
Iteration 10, Average error: 1.8483331872627828
Finishing due to change in average error less than 0.001 (proportional)
{'hyperparams': {'aggregation_topscores_p': 0.5,
                 'beta_alpha': 10,
                 'beta_beta': 1.25,
                 'crosssvalidation_splits': 5,
                 'default_rating': 10,
                 'error_change_prop': 0.001,
                 'finish_early': False,
                 'linear_mean': 10,
                 'max_iter': 50,
                 'test_set_size': 0.2,
                 'truncexp_base_mean': 10,
                 'truncexp_max': 100},
 'results': {'test_error'

Crossvalidation run 1
Iteration 1, Average error: 1.8000586245367485
Iteration 2, Average error: 1.583476576594782
Iteration 3, Average error: 1.5500520140439435
Iteration 4, Average error: 1.5466423400722114
Iteration 5, Average error: 1.5489840162001194
Iteration 6, Average error: 1.5517568920582068
Iteration 7, Average error: 1.5538911246440301
Iteration 8, Average error: 1.5553477490984893
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 2
Iteration 1, Average error: 1.8037978503493117
Iteration 2, Average error: 1.588055593768726
Iteration 3, Average error: 1.5540360462733183
Iteration 4, Average error: 1.5502795564087377
Iteration 5, Average error: 1.5525101429548371
Iteration 6, Average error: 1.5552693839578957
Iteration 7, Average error: 1.557405548445906
Iteration 8, Average error: 1.5588828671906805
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 3
Iteration 1, Average error: 1.81138291789

Iteration 1, Average error: 1.7074256469116853
Iteration 2, Average error: 1.5241853982849234
Iteration 3, Average error: 1.4927506927655476
Iteration 4, Average error: 1.4881260317889702
Iteration 5, Average error: 1.48899412634151
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 1.6973265060827358
Iteration 2, Average error: 1.514908267112823
Iteration 3, Average error: 1.483054635471108
Iteration 4, Average error: 1.4782800396776437
Iteration 5, Average error: 1.479148724211291
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 2
Iteration 1, Average error: 1.7094498575397972
Iteration 2, Average error: 1.52438003201187
Iteration 3, Average error: 1.4926172140757603
Iteration 4, Average error: 1.4880142298895027
Iteration 5, Average error: 1.4890054628492815
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 3
Iteration 1, Average

In [39]:
pprint.pprint(hs_results,width=1)

[{'problem_params': {'error_aggregation_bottomscores_p': 0.25},
  'runs': [{'hyperparams': {'aggregation_topscores_p': 0.25,
                            'beta_alpha': 10,
                            'beta_beta': 1.25,
                            'crosssvalidation_splits': 5,
                            'default_rating': 10,
                            'error_change_prop': 0.005,
                            'finish_early': False,
                            'linear_mean': 10,
                            'max_iter': 50,
                            'test_set_size': 0.2,
                            'truncexp_base_mean': 10,
                            'truncexp_max': 100},
            'results': {'test_error': 1.0283248730593837,
                        'train_error': 0.5790266175302495}},
           {'hyperparams': {'aggregation_topscores_p': 0.25,
                            'beta_alpha': 10,
                            'beta_beta': 1.25,
                            'crosssvalidation_spl

In [43]:
with open('hyperparam_search_results_20241104.txt', 'w') as f:
    pprint.pprint(hs_results, width=1, stream=f)

In [40]:
for hs_problem in hs_results:
    hs_problem_params = hs_problem["problem_params"]
    hs_runs = hs_problem["runs"]
    
    min_test_error = math.inf
    min_test_error_hyperparams = None
    min_test_error_results = None
    
    for hs_run in hs_runs:
        hs_hyperparams = hs_run["hyperparams"]
        hs_run_results = hs_run["results"]
        hs_test_error = hs_run_results["test_error"]
        
        if hs_test_error < min_test_error:
            min_test_error = hs_test_error
            min_test_error_hyperparams = hs_hyperparams
            min_test_error_results = hs_run_results
    
    print(f"The run with the lowest validation error for problem {hs_problem_params} is the following:")
    pprint.pprint(min_test_error_hyperparams,width=1)
    pprint.pprint(min_test_error_results,width=1)

The run with the lowest validation error for problem {'error_aggregation_bottomscores_p': 0.25} is the following:
{'aggregation_topscores_p': 0.9,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.005,
 'finish_early': False,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 0.5599339294446171,
 'train_error': 0.3008765915976855}
The run with the lowest validation error for problem {'error_aggregation_bottomscores_p': 0.5} is the following:
{'aggregation_topscores_p': 0.9,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.005,
 'finish_early': False,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 1.2019101868931037,
 'train_error': 0.6591875257636912}
The run with the lowest validation error for problem {'

In [41]:
for hs_problem in hs_results:
    hs_problem_params = hs_problem["problem_params"]
    hs_runs = hs_problem["runs"]
    
    min_train_error = math.inf
    min_train_error_hyperparams = None
    min_train_error_results = None
    
    for hs_run in hs_runs:
        hs_hyperparams = hs_run["hyperparams"]
        hs_run_results = hs_run["results"]
        hs_train_error = hs_run_results["train_error"]
        
        if hs_train_error < min_train_error:
            min_train_error = hs_train_error
            min_train_error_hyperparams = hs_hyperparams
            min_train_error_results = hs_run_results
    
    print(f"The run with the lowest training error for problem {hs_problem_params} is the following:")
    pprint.pprint(min_train_error_hyperparams,width=1)
    pprint.pprint(min_train_error_results,width=1)

The run with the lowest training error for problem {'error_aggregation_bottomscores_p': 0.25} is the following:
{'aggregation_topscores_p': 0.9,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.0002,
 'finish_early': False,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 0.5783391797412771,
 'train_error': 0.29979437523152197}
The run with the lowest training error for problem {'error_aggregation_bottomscores_p': 0.5} is the following:
{'aggregation_topscores_p': 0.9,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.001,
 'finish_early': False,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 1.2406950253499243,
 'train_error': 0.6559652638382691}
The run with the lowest training error for problem {'erro

## Basic result exploration

In [20]:
map_outliers = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 3}
player_outliers = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 30}

In [21]:
print(map_outliers)
print(player_outliers)

{'2dd6cxx92': 2.0152677292713266, '36bce91': 2.4800652588369436, '3442cxxx91': 2.718401184329409, '3697axxxx91': 2.0782249943316957, '3aa79xxxxxxxxx91': 1.9067157917596442, '3b2bcxxxx91': 2.3575356965841343, '3b5a9xxxx92': 2.395904890598641, '41f391': 2.778534676780568, '6b5f71': 2.711668884778338}
{'3225556157461414': 30.29976392203595, '76561198180044686': 30.445249373294917, '1922350521131465': 34.72475456499676, '76561198988695829': 32.0349673851341, '76561199085118735': 30.09488546851708, '76561198166061709': 33.48055170841957, '76561198404774259': 30.10269148198995, '76561198333869741': 30.863387386727954, '76561199465530115': 31.821988743620192}


In [75]:
hard_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 2}
print(hard_maps)

{'3bcf5xxxxxxxx91': 1.9239252534840148, '3697axxxx91': 1.879506780858789, '3aa79xxxxxxxxx91': 1.5321603437513578, '3e2c2xxxxx91': 1.611314005933578, '2dd6cxx92': 1.6227539006198541, '3ce7axxxxxxxxxxx91': 1.5221593121674297}


In [76]:
easy_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating > 30}
print(easy_maps)

{'2a57111': 31.394432264681935, '3bb99x11': 30.946497598534165, '7e8f11': 30.958264493732337}


In [77]:
good_players = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 32}
print(good_players)

{'76561198333869741': 32.08228910252549, '76561198988695829': 32.388853873720784, '1922350521131465': 35.90909358460172, '76561198166061709': 35.29837900080887, '76561199465530115': 33.1591378916513}


In [94]:
#print(scores_doubly_indexed["7e8f11"])
print(maps_by_id["3ad60xxxxxx11"])
#print(players_by_id["76561198333869741"])

{'hash': '58ff349229bf3a466783c28e65565f7135a4f79b', 'name': "DA'AT -The First Seeker of Souls-", 'id': '3ad60xxxxxx11', 'songId': '3ad60xxxxxx', 'modeName': 'Standard', 'difficultyName': 'Easy', 'accRating': 4.5261936, 'passRating': 2.1133034, 'techRating': 1.2710434, 'predictedAcc': 0.99053746, 'modifiersRating': {'id': 1237119, 'ssPredictedAcc': 0.99136376, 'ssPassRating': 1.656638, 'ssAccRating': 4.203791, 'ssTechRating': 1.0673054, 'ssStars': 2.4335692, 'fsPredictedAcc': 0.9893439, 'fsPassRating': 2.7446237, 'fsAccRating': 4.9438214, 'fsTechRating': 1.5058012, 'fsStars': 3.2402518, 'sfPredictedAcc': 0.9872984, 'sfPassRating': 3.7513492, 'sfAccRating': 5.7244205, 'sfTechRating': 1.7997385, 'sfStars': 4.07828, 'bfsPredictedAcc': 0.98967737, 'bfsPassRating': 2.7446237, 'bfsAccRating': 4.828963, 'bfsTechRating': 1.5058012, 'bfsStars': 3.1622553, 'bsfPredictedAcc': 0.9882569, 'bsfPassRating': 3.7142072, 'bsfAccRating': 5.359308, 'bsfTechRating': 1.7819194, 'bsfStars': 3.8063192}}


In [ ]:
print(player_ratings["76561199108348236"])

In [ ]:
print(scores_by_player_id["76561199108348236"])

In [ ]:
print(map_ratings["38419x92"])

## Useful stuff for extra detailed exploration

In [ ]:
def errors_in_map(map_id,map_ratings,player_ratings):
    result = {}
    map_rating = map_ratings[map_id]
    for player_id,score in scores_doubly_indexed[map_id].items():
        player_rating = player_ratings[player_id]
        calculated_w = bps.wfun(map_rating,player_rating)
        error = bps.error_fun(calculated_w,score)
        result[player_id] = {"calculated_w":calculated_w,"actual_score":score,"error":error}
        
    return result

In [ ]:
errors_this_map = errors_in_map("1442791",maps_1,players_1)
error_outliers = {player_id:error for (player_id,error) in errors_this_map.items() if error["error"] < 1}
print(error_outliers)

In [ ]:
scores_by_map_id["2c803x91"]["76561198279631500"]

In [ ]:
hard_maps = {map_id : rating for (map_id,rating) in maps_1.items() if rating < 2}
print(hard_maps)